In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# connection to Postgres 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-12-26 00:50:34--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-12-26 00:50:35 (5.99 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

from pyspark.ml.feature import RegexTokenizer, Tokenizer
from pyspark.sql import functions as F
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Luggage_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+----------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
# Count the number of records (rows) in the dataset.
df.count()

348657

In [6]:
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'string'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'string'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'string'),
 ('total_votes', 'string'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [7]:
from pyspark.sql.functions import to_date

In [8]:
df = df.withColumn('review_date',to_date(df.review_date, 'yyyy-mm-dd'))
df = df.withColumn("customer_id",df["customer_id"].cast(IntegerType()))
df = df.withColumn("product_parent",df["product_parent"].cast(IntegerType()))
df = df.withColumn("star_rating",df["star_rating"].cast(IntegerType()))
df = df.withColumn("helpful_votes",df["helpful_votes"].cast(IntegerType()))
df = df.withColumn("total_votes",df["total_votes"].cast(IntegerType()))

In [9]:
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'date')]

In [10]:
# **Transform** the dataset to fit the tables in the [schema file](../Resources/schema.sql). Be sure the DataFrames match in data type and in column name.

In [11]:
review_id_table = df.select("review_id", "customer_id", "product_id","product_parent","review_date")
review_id_table.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R9CO86UUJCAW5|   40884699|B00VGTN02Y|     786681372| 2015-01-31|
|R3PR8X6QGVJ8B1|   23208852|B005KIWL0E|     618251799| 2015-01-31|
|R39BO2819ABUPF|   17100246|B007UNSHJ6|     810480328| 2015-01-31|
|R3ADL7V6EGGEEP|   13533670|B00WHFNXB4|      20765193| 2015-01-31|
|R1OXYPBPLVRMI5|   38541771|B000M5NBYU|     812890709| 2015-01-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [12]:
products = df.select("product_id","product_title").dropDuplicates(["product_id"])
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1933438304|Penguin Tote: I'd...|
|3924862206|Kukishinden Ryû (...|
|8862938861|Moleskine Luggage...|
|886293887X|Moleskine 3.75x2....|
|8866138274|Moleskine Luggage...|
+----------+--------------------+
only showing top 5 rows



In [13]:
customers= df.groupBy("customer_id").count().withColumnRenamed("count","customer_count")
customers.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   42596797|             1|
|   12296525|             1|
|   49101505|             1|
|   28377689|             1|
|   14468845|             1|
+-----------+--------------+
only showing top 5 rows



In [14]:
vine_table = df.select("review_id", "star_rating", "helpful_votes","total_votes","vine")
vine_table.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R9CO86UUJCAW5|          3|            0|          0|   N|
|R3PR8X6QGVJ8B1|          5|            0|          0|   N|
|R39BO2819ABUPF|          4|            0|          0|   N|
|R3ADL7V6EGGEEP|          4|            0|          0|   N|
|R1OXYPBPLVRMI5|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [15]:
# **Load** the DataFrames that correspond to tables into an RDS instance. **Note:** This process can take up to 10 minutes for each. Be sure that everything is correct before uploading.

In [16]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.ce0naysny4ho.eu-west-2.rds.amazonaws.com:5432/big_data"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}


In [17]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [18]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [19]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [20]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)